In [6]:
class Person:
    
    #i__init__ : initialize method
    # __이름__() : dundu / magic(special) method
    def __init__(self, name, age = 30): #  매직메소드
        self.name = name
        self.age = age
    
    def __str__(self): #  매직메소드
        return self.name
    
    def __call__(self): # 매직메소드
        # 객체를 함수처럼 사용 할 수 있게 해줌
        print("__call__", self.name, self.name)
        return self.age+50

In [8]:
p = Person("홍길동")
print(p.name)
print(p)
print(str(p))
print(p())

홍길동
홍길동
홍길동
__call__ 홍길동 홍길동
80
__call__ 홍길동 홍길동


80

In [5]:
str(p) # Person class의 객체인 경우ㅡ
# value -> 문자열,
# value가 객체일 경우 __str__

'홍길동'

# Sequential 모델 
- 각 Layer들의 입력과 출력이 하나라고 가정 
- 각각의 Layer(입력층, 은닉층, 출력층)들을 차례대로 쌓아 구성
- 다양한 구조의 네트워크를 만드는데 한계가 있다.

# Functional API
- 함수형 API를 사용하면 **다중입력, 다중출력, 그래프 형태**의 다양한 형태의 모델을 유연하게 구성 가능
- Functional API는 직접 텐서들의 입출력을 다룸 
- 함수호출처럼 Layer를 이용하여 입력 텐서(Input Tensor)를 입력 받고 그 결과를 출력 텐서(Output Tensor)로 반환하는 형식으로 모델 구현


```
input_tensor = Input(shape=(16,))
dense = layers.Dense(32, activation='relu')(input_tensor)
output_tensor = layers.Dense(32, activation='sigmoid')(dense)

model = models.Model(input_tensor, output_tensor)
```

### layer 구조 만들기

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# 레이어들의 구조를 먼저 구성
# 출력 변수들은 통일, 일반적으로 block 단위로 동일한 변수명 사용

input_tensor = layers.Input(shape=(32, 32, 3)) # input layer는 입력을 지정하지 않는다. (model.predict(입력))

x1 = layers.Conv2D(filters=16, kernel_size=3, padding="same", activation = "relu")(input_tensor)
x1 = layers.MaxPooling2D(padding="same")(x1)

x2 = layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu")(x1)
x2 = layers.MaxPooling2D(padding="same")(x2)

x3 = layers.Flatten()(x2)
x3 = layers.Dense(units=8, activation="relu")(x3)

output_tensor = layers.Dense(units=1, activation="sigmoid")(x3)

# 보통 중간 단계를 쓸 일이 없으면 이렇게 block 당 같은 이름으로 구성

In [10]:
# 레이어들의 구조를 먼저 만듦
# 출력 변수들은 통일, 일반적으로 block 단위로 동일한 변수명 사용

# trainable 설정
input_tensor = layers.Input(shape=(32, 32, 3))

conv_layer = layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu")
conv_layer.trainable = False

x1 = conv_layer(input_tensor)
x1 = layers.MaxPooling2D(padding="same")(x1)

x2 = layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu")(x1)
x2 = layers.MaxPooling2D(padding="same")(x2)

x3 = layers.Flatten()(x2)
x3 = layers.Dense(units=8, activation="relu")(x3)

output_tensor = layers.Dense(units=1, activation = "sigmoid")(x3)

### 모델 정의

In [11]:
fn_model = models.Model(input_tensor, output_tensor) # (입력 tensor, 출력 tensor)
fn_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 16)        2320      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 8200

#### 모델 컴파일, 모델 학습, 평가, 추론은 동일
#### 모델을 만드는 방식이 다름


<hr>

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16

In [14]:
# 레이어 만들기

conv_base = VGG16(include_top = False)

input_tensor = layers.Input(shape=(2244, 224, 3))
x = conv_base(input_tensor)
x = layers.GlobalAveragePooling2D()(x)

output_tensor = layers.Dense(units=1, activation="sigmoid")(x)

# 모델 설정
model = models.Model(input_tensor, output_tensor)
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])
model.summary()


58892288/58889256 [==============================] - 7s 0us/step
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 2244, 224, 3)]    0         
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 513       
Total params: 14,715,201
Trainable params: 14,715,201
Non-trainable params: 0
_________________________________________________________________


<hr>

#### 레이어를 합치는 함수
- concatenate(list, axis=-1)
    - 레이어들을 합친다
    - list: 합칠 레이어들을 리스트에 묶어 전달
    - axis: 합칠 기준축. (기본값: -1 : 마지막 축기준)
- add(list), substract(list), multiply(list)
    - 같은 index의 값들을 계산해서(더하기, 빼기, 곱하기) 하나의 레이어로 만든다.
    - list: 합칠 레이어들을 리스트에 묶어 전달

In [17]:
# Resnet의 residual block

input_tensor = layers.Input(shape=(32, 32, 3))

x = layers.Conv2D(filters=64, kernel_size=3, padding="same")(input_tensor)

x1 = layers.Conv2D(filters=64, kernel_size=3, padding="same")(x)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ReLU()(x1)

x2 = layers.Conv2D(filters=64, kernel_size=3, padding="same")(x1)
x2 = layers.BatchNormalization()(x2)

# input_tensor(입력), x2(block의 출력)을 더함

add_result = layers.add([x, x2])

output_tensor = layers.ReLU()(add_result)

rb_model = models.Model(input_tensor, output_tensor)


In [18]:
rb_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 32, 32, 64)   1792        input_11[0][0]                   
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 32, 32, 64)   36928       conv2d_17[0][0]                  
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 32, 32, 64)   256         conv2d_18[0][0]                  
____________________________________________________________________________________________

In [21]:
keras.utils.plot_model(rb_model, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


<hr>

## 다중 출력 모델
- 가정
    - iris 데이터셋에서 꽃받침의 너비와 높이로 꽃입의 너비, 높이, 꽃 종류를 예측하는 모델
    - 출력결과가 3개(꽃입의 너비, 높이, 꽃 종류)
- X: 꽃받침 너비, 높이
- y: 꽃잎 너비, 높이, 꽃 종류

In [23]:
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
# print(X.shape, y. shape)

y1 = X[:, 2]
y2 = X[:, 3]
y3 = y

X = X[:, [0, 1]]

(150, 4) (150,)


### 모델 정의

In [27]:
input_tensor = layers.Input(shape=(2,))

x = layers.Dense(units=16, activation="relu")(input_tensor)
x = layers.Dense(units=8, activation="relu")(x)

output_1 = layers.Dense(units=1, name = "petal_width_output")(x) # 꽃잎 너비(연속형) : 회귀 : units : 1 ,activation : None
output_2 = layers.Dense(units=1, name = "petal_length_output")(x)
output_3 = layers.Dense(units=3, activation = "softmax", name="species_output")(x) # 종류(범주형 - 다중분류) : units : class 개수, activation : softmax


model = models.Model(input_tensor, [output_1, output_2, output_3]) # input, output이 여러개일 경우 List로 묶어서 전달 

In [28]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 2)]          0                                            
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 16)           48          input_14[0][0]                   
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 8)            136         dense_10[0][0]                   
__________________________________________________________________________________________________
petal_width_output (Dense)      (None, 1)            9           dense_11[0][0]                   
____________________________________________________________________________________________

In [29]:
keras.utils.plot_model(model, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [31]:
model.compile(optimizer = "adam",
             loss = ["mse", "mse", "sparse_categorical_crossentropy"] # 출력(예측) 결과 3개에 대한 각각의 loss 함수를 리스트로 묶어서 전달 
             )

# "sparse_categorical_crossentropy" : y(정답)을 one-hot-encoding을 내부적으로 처리 후 오차계산을 해줌
# 3개의 오차를 더해서 total loss를 계산한 뒤 total loss 를 기반으로 역전파해서 파라미터들을 업데이트

In [33]:
hist = model.fit(x = X, y = [y1, y2, y3], epochs=100, validation_split=0.1)

Epoch 1/100
5/5 [==============================] - 2s 153ms/step - loss: 16.4455 - petal_width_output_loss: 10.1624 - petal_length_output_loss: 5.1245 - species_output_loss: 1.1585 - val_loss: 29.8194 - val_petal_width_output_loss: 18.8791 - val_petal_length_output_loss: 10.2883 - val_species_output_loss: 0.6520
Epoch 2/100
5/5 [==============================] - 0s 12ms/step - loss: 15.6795 - petal_width_output_loss: 9.6226 - petal_length_output_loss: 4.8946 - species_output_loss: 1.1624 - val_loss: 28.5098 - val_petal_width_output_loss: 17.9560 - val_petal_length_output_loss: 9.9060 - val_species_output_loss: 0.6478
Epoch 3/100
5/5 [==============================] - 0s 16ms/step - loss: 14.9324 - petal_width_output_loss: 9.1002 - petal_length_output_loss: 4.6649 - species_output_loss: 1.1674 - val_loss: 27.1639 - val_petal_width_output_loss: 16.9916 - val_petal_length_output_loss: 9.5307 - val_species_output_loss: 0.6416
Epoch 4/100
5/5 [==============================] - 0s 14ms/step 

5/5 [==============================] - 0s 15ms/step - loss: 4.0523 - petal_width_output_loss: 2.3308 - petal_length_output_loss: 0.5536 - species_output_loss: 1.1679 - val_loss: 4.9957 - val_petal_width_output_loss: 2.3531 - val_petal_length_output_loss: 1.7824 - val_species_output_loss: 0.8602
Epoch 28/100
5/5 [==============================] - 0s 15ms/step - loss: 3.9384 - petal_width_output_loss: 2.2737 - petal_length_output_loss: 0.4999 - species_output_loss: 1.1647 - val_loss: 4.6490 - val_petal_width_output_loss: 2.1689 - val_petal_length_output_loss: 1.6056 - val_species_output_loss: 0.8745
Epoch 29/100
5/5 [==============================] - 0s 14ms/step - loss: 3.8423 - petal_width_output_loss: 2.2267 - petal_length_output_loss: 0.4547 - species_output_loss: 1.1609 - val_loss: 4.4091 - val_petal_width_output_loss: 2.0603 - val_petal_length_output_loss: 1.4576 - val_species_output_loss: 0.8912
Epoch 30/100
5/5 [==============================] - 0s 14ms/step - loss: 3.7687 - peta

5/5 [==============================] - 0s 12ms/step - loss: 2.9891 - petal_width_output_loss: 1.5835 - petal_length_output_loss: 0.3567 - species_output_loss: 1.0488 - val_loss: 3.4096 - val_petal_width_output_loss: 1.4856 - val_petal_length_output_loss: 0.8514 - val_species_output_loss: 1.0725
Epoch 54/100
5/5 [==============================] - 0s 14ms/step - loss: 2.9637 - petal_width_output_loss: 1.5600 - petal_length_output_loss: 0.3564 - species_output_loss: 1.0472 - val_loss: 3.4319 - val_petal_width_output_loss: 1.5023 - val_petal_length_output_loss: 0.8485 - val_species_output_loss: 1.0810
Epoch 55/100
5/5 [==============================] - 0s 13ms/step - loss: 2.9391 - petal_width_output_loss: 1.5374 - petal_length_output_loss: 0.3560 - species_output_loss: 1.0458 - val_loss: 3.4512 - val_petal_width_output_loss: 1.5129 - val_petal_length_output_loss: 0.8469 - val_species_output_loss: 1.0914
Epoch 56/100
5/5 [==============================] - 0s 16ms/step - loss: 2.9170 - peta

5/5 [==============================] - 0s 15ms/step - loss: 2.4168 - petal_width_output_loss: 1.0502 - petal_length_output_loss: 0.3379 - species_output_loss: 1.0287 - val_loss: 3.1199 - val_petal_width_output_loss: 1.1759 - val_petal_length_output_loss: 0.8088 - val_species_output_loss: 1.1352
Epoch 80/100
5/5 [==============================] - 0s 16ms/step - loss: 2.3965 - petal_width_output_loss: 1.0323 - petal_length_output_loss: 0.3364 - species_output_loss: 1.0278 - val_loss: 3.0874 - val_petal_width_output_loss: 1.1482 - val_petal_length_output_loss: 0.7984 - val_species_output_loss: 1.1408
Epoch 81/100
5/5 [==============================] - 0s 21ms/step - loss: 2.3777 - petal_width_output_loss: 1.0162 - petal_length_output_loss: 0.3345 - species_output_loss: 1.0270 - val_loss: 3.0875 - val_petal_width_output_loss: 1.1488 - val_petal_length_output_loss: 0.7923 - val_species_output_loss: 1.1464
Epoch 82/100
5/5 [==============================] - 0s 16ms/step - loss: 2.3581 - peta

In [37]:
# new_dat
new_data = X[:3]
pred = model.predict(new_data)

In [36]:
pred

[array([[2.501481 ],
        [2.7471495],
        [2.3343337]], dtype=float32),
 array([[0.90332043],
        [0.9480382 ],
        [0.8646666 ]], dtype=float32),
 array([[0.35390592, 0.3807085 , 0.26538554],
        [0.33470297, 0.38593957, 0.27935746],
        [0.3568602 , 0.37842163, 0.26471815]], dtype=float32)]

In [38]:
print(y1[0])
print(y2[0])
print(y3[0])

1.4
0.2
0


## 다중 입력 모델
- 가정 
    - IRIS 꽃 데이터 + 꽃의 사진을 입력해서 꽃의 종류를 예측
- X: 꽃 데이터, 꽃 사진
- y: 꽃 종류

In [39]:
# iris 사진 대신 mnist 사진 사용 (예시니까...)
X, y = load_iris(return_X_y=True)
X.shape , y.shape

((150, 4), (150,))

In [40]:
(X_train, _), (_,_) = keras.datasets.mnist.load_data()
X_train.shape
X_img = X_train[:150]
X_train.shape, X_img.shape

((60000, 28, 28), (150, 28, 28))

In [43]:
# input을 2개 받는 모델 구현

iris_info_tensor = layers.Input(shape=(4,)) # 꽃 정보
x1 = layers.Dense(units=32, activation="relu")(iris_info_tensor)
x1 = layers.Dense(units=16, activation="relu")(x1)

iris_img_tensor = layers.Input(shape=(28, 28, 1))
x2 = layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")(iris_img_tensor)
x2 = layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")(x2)
x2 = layers.MaxPooling2D(padding="same")(x2)

x3 = layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu")(x2)
x3 = layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu")(x3)
x3 = layers.MaxPooling2D(padding="same")(x3)
x3 = layers.GlobalAveragePooling2D()(x3)

x4 = layers.concatenate([x1, x3])

output_tensor = layers.Dense(units=3, activation="softmax")(x4)

model = models.Model([iris_info_tensor, iris_img_tensor],output_tensor)

In [44]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 28, 28, 32)   320         input_20[0][0]                   
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 28, 28, 32)   9248        conv2d_28[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_14 (MaxPooling2D) (None, 14, 14, 32)   0           conv2d_29[0][0]                  
____________________________________________________________________________________________

In [45]:
keras.utils.plot_model(model, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [46]:
model.compile(optimizer="adam", loss = "sparse_categorical_crossentropy", metrics=['accuracy'])

In [48]:
model.fit(x = [X, X_img], y = y, epochs=100, validation_split=0.1)

Epoch 1/100
5/5 [==============================] - 1s 117ms/step - loss: 1.0359 - accuracy: 0.5185 - val_loss: 1.4585 - val_accuracy: 0.0000e+00
Epoch 2/100
5/5 [==============================] - 1s 109ms/step - loss: 0.9617 - accuracy: 0.6222 - val_loss: 1.8222 - val_accuracy: 0.0000e+00
Epoch 3/100
5/5 [==============================] - 0s 92ms/step - loss: 0.9331 - accuracy: 0.7259 - val_loss: 1.4186 - val_accuracy: 0.0000e+00
Epoch 4/100
5/5 [==============================] - 0s 98ms/step - loss: 0.8746 - accuracy: 0.7259 - val_loss: 1.1192 - val_accuracy: 0.0000e+00
Epoch 5/100
5/5 [==============================] - 0s 93ms/step - loss: 0.8495 - accuracy: 0.7333 - val_loss: 1.0568 - val_accuracy: 0.0000e+00
Epoch 6/100
5/5 [==============================] - 1s 110ms/step - loss: 0.8199 - accuracy: 0.7185 - val_loss: 1.1423 - val_accuracy: 0.0000e+00
Epoch 7/100
5/5 [==============================] - 0s 93ms/step - loss: 0.7892 - accuracy: 0.6889 - val_loss: 1.2298 - val_accuracy: 

5/5 [==============================] - 0s 94ms/step - loss: 0.1448 - accuracy: 0.9259 - val_loss: 0.3387 - val_accuracy: 0.8000
Epoch 59/100
5/5 [==============================] - ETA: 0s - loss: 0.0831 - accuracy: 0.98 - 0s 92ms/step - loss: 0.0806 - accuracy: 0.9852 - val_loss: 3.1796 - val_accuracy: 0.0667
Epoch 60/100
5/5 [==============================] - 0s 95ms/step - loss: 0.1331 - accuracy: 0.9556 - val_loss: 0.8158 - val_accuracy: 0.4667
Epoch 61/100
5/5 [==============================] - 0s 95ms/step - loss: 0.0793 - accuracy: 0.9852 - val_loss: 1.6969 - val_accuracy: 0.2667
Epoch 62/100
5/5 [==============================] - 0s 93ms/step - loss: 0.0521 - accuracy: 1.0000 - val_loss: 0.2462 - val_accuracy: 0.9333
Epoch 63/100
5/5 [==============================] - 0s 102ms/step - loss: 0.0525 - accuracy: 1.0000 - val_loss: 2.6124 - val_accuracy: 0.1333
Epoch 64/100
5/5 [==============================] - 1s 109ms/step - loss: 0.0830 - accuracy: 0.9704 - val_loss: 0.5863 - val

In [49]:
tf.data.Dataset.from_tensor_slices(((X, X_img), y))

<TensorSliceDataset shapes: (((4,), (28, 28)), ()), types: ((tf.float64, tf.uint8), tf.int32)>